### Single course with multiple iClicker sections

* Optional - Excused miss excel worksheet
    In my course, we have excused miss sheet which can track students' missed class in each class day.

In [12]:
# Optional
import pandas as pd
import os

def excused_miss_sheet(path=str, class_day=list, class_begin=str, class_end=str):
    # path includes excel file name and extension: "PATH/FNAME.xlsx"
    # class begin or end format: "YYYY-MM-DD"
    df = pd.DataFrame(pd.date_range(class_begin, class_end, freq='D').to_list(),columns=['date'])
    df['weekday'] = df['date'].dt.day_name()
    df = df.loc[df['weekday'].isin(class_day)].reset_index(drop=True)
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    df.to_excel(os.path.abspath(path))

excused_miss_sheet('excusedmiss.xlsx', ['Monday','Wednesday','Friday'] ,'2023-08-21', '2023-12-15')

#### Step 1. Loading Canvas gradebook.

In [ ]:
import pandas as pd
import os
import numpy as np
import glob
import re
import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_rows', 100)

# 
# Loading the latest canvas gradebook that saved in a "Gradebook" directory.
f_canvas        = sorted(glob.glob('Gradebook/*.csv'), key=os.path.getatime, reverse=True)[0]
df_canvas       = pd.read_csv(f_canvas, dtype={'ID':str, 'SIS User ID':str}).dropna(subset=['SIS User ID']).sort_values(['Section', 'Student'])
# set a short section name for convinience
df_canvas['Section_short'] = df_canvas['Section'].str[-1:]


# Use following info kept over the program.
var_info               = ['Student', 'ID', 'SIS User ID','SIS Login ID','Section']
# We use two metrics: Correct answer rate and Participation rate.
var_correct_rate       = df_canvas.filter(like='iClicker correct answer rate').columns.values[0]
var_participation_rate = df_canvas.filter(like='iClicker participation rate').columns.values[0]

# Subsetting only infos for convinience.
df_canvas_info = df_canvas[['Student', 'ID', 'SIS User ID','SIS Login ID','Section_short']].rename(columns={'Section_short':'Section'}).reset_index(drop=True).copy()


df_canvas_info
df_canvas

Step 2. Loading class records of iClicker

In [ ]:
# Class history file is automatically generated when you downloading class record with my code. 
# It contains class sequence and its date.
df_class_history = {}

# 
for idx, i_hist in enumerate(sorted(glob.glob(r'iClicker\class_history* sec *.xlsx'), key=os.path.getmtime, reverse=True)[:3][::-1]) :
    df_class_history[idx] = pd.read_excel(i_hist).drop(columns=['Unnamed: 0']).sort_values(by=['date']).reset_index(drop=True)
    df_class_history[idx]['poll_sec_{0}'.format(idx+1)] = df_class_history[idx]['poll_sec_{0}'.format(idx+1)].str.strip()
    
# df_class_history[1]



# loading the newest iClciker roster of three sections
lst_last_3 = sorted(glob.glob(r'iClicker/roster iClicker *.csv'), key=os.path.getmtime, reverse=True)[:3][::-1]
print(lst_last_3)

# appding 3 sections
df_iclicker = pd.DataFrame()
for idx, i_file in enumerate(lst_last_3) :
    print(idx, i_file, i_file[-5:-4])
    # preliminary import
    df_i = pd.read_csv(i_file).rename(columns=lambda x: re.sub(' [0-9]+ -','',x))
    # get the session info
    dtype_info = dict(zip(df_i.filter(like='Class').columns.values, ['int']*len(df_i.filter(like='Class').columns.values)))
    df_i = pd.read_csv(i_file, keep_default_na = True, na_values = 'No Response', dtype = dtype_info)
    df_i['Section'] = i_file[-5:-4]
    #     
    # Column name change with class history
    df_class_history_i = df_class_history[idx][['date', 'poll_sec_{0}'.format(i_file[-5:-4])]].dropna(how='any')
    lst_col_name = dict(zip(df_class_history_i['poll_sec_{0}'.format(i_file[-5:-4])].to_list(), df_class_history_i['date']))
    # 
    df_i = df_i.rename( columns= dict(lst_col_name))
    # calculate engage
    df_i['engage'] = df_i.filter(like='2023-').count(axis=1)
    # summing duplicated dates.
    df_i = df_i.groupby(df_i.columns, axis=1).sum()
    # 
    df_iclicker = pd.concat([df_iclicker,df_i], axis=0)

df_iclicker
df_iclicker['Student'] = df_iclicker['Name']
df_iclicker['Current Score'] = df_iclicker['Performance']
df_iclicker['Student ID'] = df_iclicker['Student ID'].str.lower().str.strip()

# save "Possible Point" filter out from students list
df_iclicker_max_point = df_iclicker.loc[df_iclicker['Student'] == "Possible Points"].reset_index(drop=True)

# sorting
df_iclicker = df_iclicker.loc[df_iclicker['Student'] != "Possible Points"].dropna(subset=['Email']).sort_values(['Section','Student','Student ID']).reset_index(drop=True)

# pulling student #
df_iclicker['input 0 SIS User ID'] = np.where(df_iclicker['Student ID'].str.match('[0-9]'), df_iclicker['Student ID'], np.NaN)
# pulling paw from email
df_iclicker['input 1 SIS User Login'] = np.where(df_iclicker['Email'].str.match('[a-z0-9]+@[a-z.]+'), df_iclicker['Email'].str.replace('@[a-z.]+',''), np.NaN)
# pulling paw or student # from student input
df_iclicker['input 2 SIS User Login'] = np.where(df_iclicker['Student ID'].str.match('[a-z0-9]'), df_iclicker['Student ID'], np.NaN)

# merging paw with student #
df_iclicker = df_iclicker.merge(df_canvas_info[['SIS User ID','SIS Login ID']].rename(columns={'SIS Login ID':'SIS Login ID 0'}), left_on=['input 0 SIS User ID'], right_on=['SIS User ID'], how='left').drop(columns=['SIS User ID'])
# merging paw with from email
df_iclicker = df_iclicker.merge(df_canvas_info[['SIS Login ID']].rename(columns={'SIS Login ID':'SIS Login ID 1'}), left_on=['input 1 SIS User Login'], right_on=['SIS Login ID 1'], how='left')
# merging paw with student input
df_iclicker = df_iclicker.merge(df_canvas_info[['SIS Login ID']].rename(columns={'SIS Login ID':'SIS Login ID 2'}), left_on=['input 2 SIS User Login'], right_on=['SIS Login ID 2'], how='left')

# construct paw from mergings.
df_iclicker['SIS Login ID'] = np.NaN
df_iclicker['SIS Login ID'] = df_iclicker[['SIS Login ID 0', 'SIS Login ID 1', 'SIS Login ID 2','SIS Login ID']].fillna(method='ffill',axis=1)['SIS Login ID']

df_iclicker

In [ ]:
# Extra steps for a student who has multiple accounts

# keep the backfilled recored.
df_iclicker_paw_fixed_dup = df_iclicker[df_iclicker.duplicated(['SIS Login ID'],False)].sort_values(['SIS Login ID','Current Score'], ascending=False).reset_index(drop=True).groupby(['SIS Login ID']).fillna(method='backfill', axis=0, limit=1).merge(df_iclicker[df_iclicker.duplicated(['SIS Login ID'],False)][['Student', 'SIS Login ID']].drop_duplicates().reset_index(drop=True), how='inner', on='Student').drop_duplicates(['SIS Login ID'],keep='first')
df_iclicker_paw_fixed_dup

# unpuplicated iclicker
df_iclicker_paw_undup = df_iclicker[~df_iclicker.duplicated(['SIS Login ID'],False)].append(df_iclicker_paw_fixed_dup)
df_iclicker_paw_undup = df_iclicker_paw_undup.sort_values(['Section', 'Student']).reset_index(drop=True)
df_iclicker_paw_undup

#### Step 3. Generate grades

In [ ]:
df_iclicker_info = df_iclicker_paw_undup[['SIS Login ID', 'Section']]

# rebuild Current score
df_iclicker_info['Current Score'] = df_iclicker_paw_undup.filter(like='2023-').sum(axis=1)

# count response
df_iclicker_info['engage'] = df_iclicker_paw_undup['engage']

# set max response
df_iclicker_max_point['Max engage'] = df_iclicker_max_point.filter(like='2023-').count(axis=1)
df_iclicker_info = df_iclicker_info.merge(df_iclicker_max_point[['Current Score','Section', 'Max engage']].rename(columns={'Current Score':'Max Score'}), on='Section')

df_iclicker_info

Step 4. Adjust maximum grade with excused misses.

In [ ]:
# loading excuses list
df_excuses = pd.read_excel('excusedmiss.xlsx', sheet_name = 'Sheet2', usecols='B:ZZ').drop(columns='date').T.reset_index()
header = df_excuses.iloc[0]
df_excuses.columns = df_excuses.iloc[0]
df_excuses = df_excuses[1:].dropna(subset='input paw')
df_excuses



df_excuses['SIS Login ID'] = df_excuses['SIS Login ID'].str.lower()
df_excuses
for idx, stu in df_excuses.iterrows():
    print(idx, stu[1])
    try :
        # get section info
        # df_iclicker_info
        df_stu = df_excuses.loc[df_excuses['SIS Login ID'] == stu[1]].reset_index(drop=True)
        df = df_stu.filter(like = '2023-').T
        df.index.name = None
        lst_cols_for_count = list(df.loc[df[0] != 1].index)
        # for max score counting
        # set max response
        info_section = df_iclicker_info.loc[df_iclicker_info['SIS Login ID'] == stu[1]]['Section'].reset_index(drop=True)[0]
        df_tt_max = df_iclicker_max_point.loc[df_iclicker_max_point['Section'] == info_section][df_iclicker_max_point.columns & lst_cols_for_count].reset_index(drop=True)
        max_engage = df_tt_max.count(axis=1).min()
        # set max score
        max_score = df_tt_max.sum(axis=1)[0]
        
        # for current score counting
        df_tt_curr = df_iclicker_paw_undup.loc[df_iclicker_paw_undup['SIS Login ID'] == stu[1]].filter(like='2023-')
        # recount current response
        current_engage = df_iclicker_paw_undup.loc[df_iclicker_paw_undup['SIS Login ID'] == stu[1]]['engage'].item()
        # recount current score
        current_score = df_tt_curr[df_tt_curr.columns & lst_cols_for_count].reset_index(drop=True).sum(axis=1)[0]
        # 
        print('found', 'enage_max',max_engage, 'engage', current_engage,'score_max',max_score, 'score',current_score )
        df_iclicker_info.loc[df_iclicker_info['SIS Login ID'] == stu[1], 'Max engage'] = max_engage
        df_iclicker_info.loc[df_iclicker_info['SIS Login ID'] == stu[1], 'Max Score'] = max_score
        df_iclicker_info.loc[df_iclicker_info['SIS Login ID'] == stu[1], 'engage'] = current_engage
        df_iclicker_info.loc[df_iclicker_info['SIS Login ID'] == stu[1], 'Current Score'] = current_score
    except :
        print('fail')
        continue
    

# adding stats
df_iclicker_info[var_correct_rate] = round(df_iclicker_info['Current Score'] / df_iclicker_info['Max Score'] * 100)
df_iclicker_info[var_correct_rate] = df_iclicker_info[var_correct_rate].apply(lambda x: min(x, 100))
df_iclicker_info['iClicker correct response counter'] = df_iclicker_info['Current Score']
df_iclicker_info[var_participation_rate] = round(df_iclicker_info['engage'] / df_iclicker_info['Max engage'] * 100)
df_iclicker_info[var_participation_rate] = df_iclicker_info[var_participation_rate].apply(lambda x: min(x, 100))
# Extra credit - when you need.
# df_iclicker_info['Extra credit'] = np.where( df_iclicker_info['iClicker correct answer rate (1439561)'] >= 50, 1,0)
df_iclicker_info


# Exportable iClicker - bring correct name from canvas 
df_iclicker_info_exp = df_canvas[['Student', 'ID', 'SIS User ID','SIS Login ID','Section', 'Section_short']].merge(df_iclicker_info.rename(columns={'Section':'Section_short'}), on=['SIS Login ID','Section_short'], how='left').fillna(0)
df_iclicker_info_exp

Step 5. Export to Excel and for Canvas import

In [ ]:
df_iclicker_info_exp.to_excel('iClicker/iClicker - gradebook {:%m-%d-%y}.xlsx'.format(datetime.datetime.now()))
df_iclicker_info_exp[var_info + [var_correct_rate, var_participation_rate]].to_csv('iClicker/iClicker - canvas import {:%m-%d-%y}.csv'.format(datetime.datetime.now()), index=False)